In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

In [2]:
def convert_rating(row):
    row[2] = float(row[2])
    return row

path = '/Users/shijialiang/data/amazon/ratings_Kindle_Store.csv'
books_data = sc.textFile(path).map(lambda x: x.split(',')).map(lambda x: convert_rating(x)).cache()

from pyspark.sql.types import *
schema = StructType([StructField("user", StringType(), False),
                      StructField("item", StringType(), False),
                      StructField("rating", FloatType(), False),
                      StructField("time", StringType(), True)
                    ])

books_df = ss.createDataFrame(books_data, schema).cache()

In [3]:
books_df.count()

3205467

In [4]:
books_df.show(5)

+--------------+----------+------+----------+
|          user|      item|rating|      time|
+--------------+----------+------+----------+
|A2GZ9GFZV1LWB0|1603420304|   4.0|1405209600|
|A1K7VSUDCVAPW8|1603420304|   3.0|1282176000|
|A35J5XRE5ZT6H2|1603420304|   4.0|1365206400|
|A3DGZNFSMNWSX5|1603420304|   4.0|1285632000|
|A2CVDQ6H36L4VL|1603420304|   5.0|1342396800|
+--------------+----------+------+----------+
only showing top 5 rows



In [5]:
books_df.write.saveAsTable('kindle')

In [6]:
# number of users
ss.sql('select distinct user from kindle').count()

1406890

In [7]:
ss.sql('select distinct user from kindle group by user having count(item) > 100').count()

1229

In [8]:
# number of books
ss.sql('select distinct item from kindle').count()

430530

In [ ]:
ss.sql('select distinct item from kindle group by item having count(user) > 100').count()

In [15]:
ss.sql('select distinct item from kindle group by item having count(user) > 2000').show()

+----------+
|      item|
+----------+
|B00CATSONE|
|B006GWO5NE|
|B006P5CH1O|
|B006GWO5WK|
|B005DOK8NW|
|B006YC7A0Q|
|B007R5YDYA|
|B00BTIDW4S|
|B00825BZUY|
|B00902SFC4|
|B0093MU7QS|
+----------+



In [10]:
# filtered data columns
ss.sql('select * from kindle\
        where user in (select distinct user from kindle group by user having count(item) > 100)\
          and item in (select distinct item from kindle group by item having count(user) > 60)')\
.count()

39470